Notebook to generate adjacency matrices of our scripts in the juliet dataset to be used as input for our neural network model.

In [253]:
import ast
import pickle
import numpy as np
import pandas as pd
from scipy import sparse
import networkx as nx
from preprocess_code import *

In [255]:
data = pd.read_csv("../data/buffer_overflow_data.csv.gz")

In [256]:
data = data.iloc[0:500]
# data = data.iloc[0:100]

In [4]:
def generate_edge_list1(testcase, **kwargs):
    """
    Takes in a list of files/datapoints from juliet.csv.zip 
    or (as loaded with pandas) matching one particular testcase, 
    and returns an edge list of its graph representation.
    """
    parse_list = [
        (datapoint.filename, datapoint.code)
        for datapoint in testcase.itertuples()
    ]

    primary = find_primary_source_file(testcase)

    # Parse the source code with clang, and get out an ast:
    index = clang.cindex.Index.create()
    translation_unit = index.parse(
        path=primary.filename,
        unsaved_files=parse_list,
    )
    ast_root = translation_unit.cursor

    # Memoise/concretise the ast so that we can consistently
    # modify it, then number each node in the tree uniquely.
    concretise_ast(ast_root)
    number_ast_nodes(ast_root)

    # Next, construct an edge list for the graph2vec input:
    edgelist = generate_edgelist(ast_root)
    
    edgelist_representation = {
        "edges": edgelist,
    }

    # Explicitly delete clang objects
    del translation_unit
    del ast_root
    del index

    return json.dumps(edgelist_representation)

In [ ]:
# dask_data = dd.from_pandas(data, npartitions=20)

# generate the graphs for all the testcases in the dataset 

graphs = data.groupby(['testcase_ID']).apply(
        generate_edge_list1,
        axis='columns',
        meta=('generate_edge_list', 'unicode'),
    )

NameError: name 'generate_edge_list1' is not defined

In [6]:
def gen_adj_matrix1(testcase):
    
    """
    Takes in a list of files/datapoints from buffer_overflow_data.csv.gz 
    matching one particular testcase, and generates an adjacency matrix 
    from the edgelist created.
    """
    
    # extracting the list of edges 

    x = testcase.split('edges": ')
    x = x[1].split('}')
    x = ast.literal_eval(x[0])
    
#     return x

    # generating the matrix
    
    G = nx.Graph()

    G.add_edges_from(x)

    A = nx.adjacency_matrix(G)

    B = A.todense()

    return B

In [7]:
# create a dataframe containing the testcase ID and its adjacency matrix 
adjacency_df = pd.DataFrame()

In [8]:
adjacency_df['testcase_ID'] = data.testcase_ID.drop_duplicates()

In [9]:
# kernel dies when there are more than 200 datapoints

# adj_matrices = graphs.apply(gen_adj_matrix1, meta = ('generate_adj_matrices', 'O'))
adj_matrices = graphs.apply(gen_adj_matrix1)

In [10]:
# adj_matrices = pd.DataFrame(adj_matrices)
adj_matrices = adj_matrices.to_frame()

In [11]:
## TODO: in a DASK framework reset_index is not a recognized function like pandas, fix this bug

# adj_matrices = adj_matrices.compute()
adj_matrices = adj_matrices.reset_index(level='testcase_ID')

In [12]:
adjacency_df['adj_matrix'] = adj_matrices[0]

In [13]:
adj_df = adjacency_df.dropna()

In [14]:
adj_df.to_csv("../data/adj_df.csv.gz")

## Feature Matrix

In [257]:
def concretise_ast(node):
    """
    Everytime you run .get_children() on a clang ast node, it
    gives you new objects. So if you want to modify those objects
    they will lose their changes everytime you walk the tree again.
    To avoid this problem, concretise_ast walks the tree once,
    saving the resulting list from .get_children() into a a concrete
    list inside the .children.
    You can then use .children to consistently walk over tree, and
    it will give you the same objects each time.
    """
    node.children = list(node.get_children())

    for child in node.children:
        counter = concretise_ast(child)

def number_ast_nodes(node, counter=1):
    """
    Given a concretised clang ast, assign each node with a unique
    numerical identifier. This will be accessible via the .identifier
    attribute of each node.
    """
    node.identifier = counter
    counter += 1

    node.children = list(node.get_children())
    for child in node.children:
        counter = number_ast_nodes(child, counter)

    return counter


def generate_ast_roots(testcase, **kwargs):
    """
    Takes in a list of files/datapoints from juliet.csv.zip (as loaded with pandas) matching one particular
    testcase, and preprocesses it ready for the feature matrix.
    """
    
    parse_list = [
        (datapoint.filename, datapoint.code)
        for datapoint in testcase.itertuples()
    ]

    primary = find_primary_source_file(testcase)

    # Parse the source code with clang, and get out an ast:
    index = clang.cindex.Index.create()
    translation_unit = index.parse(
        path=primary.filename,
        unsaved_files=parse_list,
    )
    ast_root = translation_unit.cursor
    
    concretise_ast(ast_root)
    number_ast_nodes(ast_root)
    
    return ast_root

In [258]:
ast_roots = data.groupby(['testcase_ID']).apply(generate_ast_roots)

In [259]:
# example_node = ast_roots.iloc[0].children[19]
# dir(example_node)

Getting the columns for the feature matrix:

In [260]:
def generate_colnames(ast_root):
    """
    Given a concretised & numbered clang ast, returns a set of node kinds to be used as columns in feature matrix
    """
    features =  set()


    def walk_tree_and_set_features(node):
        out_degree = len(node.children)
        in_degree = 1
        degree = out_degree + in_degree
        
        features.add(str(node.kind))

#         features[node.identifier] = [str(node.kind)]

        for child in node.children:
            walk_tree_and_set_features(child)

    walk_tree_and_set_features(ast_root)

    return features


def generate_spelling(ast_root):
    """
    Given a concretised & numbered clang ast, returns a set of node spellings to be used later
    in constructing the columns in feature matrix
    """
    spelling =  set()


    def walk_tree_and_set_features(node):
        out_degree = len(node.children)
        in_degree = 1
        degree = out_degree + in_degree
        
        spelling.add(node.spelling)

        for child in node.children:
            walk_tree_and_set_features(child)

    walk_tree_and_set_features(ast_root)

    return spelling

Creating unique set of node kinds and node spellings

In [261]:
colnames = ast_roots.apply(generate_colnames)
spelling = ast_roots.apply(generate_spelling)

obtaining final boolean column names

In [262]:
final_colnames = set()
final_colnames.update(['WriteToPointer', 'SizeOf', 'Alloc'])
for i in range(len(feature_sets)):
    final_colnames.update(feature_sets.iloc[i])

Set of all node spellings

In [263]:
final_spelling = set()
for i in range(len(spelling)):
    final_spelling.update(spelling.iloc[i])

In [264]:
final_colnames = pd.Series(list(final_colnames))

In [289]:
final_colnames

0                       CursorKind.PACK_EXPANSION_EXPR
1                                 CursorKind.DECL_STMT
2                                   CursorKind.IF_STMT
3                              CursorKind.TYPEDEF_DECL
4                      CursorKind.CXX_STATIC_CAST_EXPR
5                 CursorKind.CXX_NULL_PTR_LITERAL_EXPR
6                            CursorKind.UNEXPOSED_EXPR
7                       CursorKind.OVERLOADED_DECL_REF
8                            CursorKind.CLASS_TEMPLATE
9                            CursorKind.STRING_LITERAL
10                          CursorKind.TYPE_ALIAS_DECL
11                             CursorKind.VARIABLE_REF
12                               CursorKind.LABEL_STMT
13                               CursorKind.CLASS_DECL
14                           CursorKind.CXX_THROW_EXPR
15                               CursorKind.UNION_DECL
16                          CursorKind.USING_DIRECTIVE
17                                CursorKind.NAMESPACE
18        

Manually pick out important node spellings 

In [239]:
alloc_list = ['__builtin_alloca', 
              '__alloc', 
              'malloc', 
              'valloc', 
              '__alloc_on_copy', 
              '__alloc_on_move', 
              'calloc', 
              'realloc', 
              'alloca',
              'ALLOCA'
             ]

sizeOf_list = ['std::aligned_storage<sizeof(_Tp), __alignof(_Tp)>'
              ]

writeToPointer_list = ['__builtin_memmove', 
                       '__builtin_memcpy', 
                       'wmempcpy', 
                       'wmemmove'
                      ]

In [273]:
# [feature for feature in final_spelling if 'Alloc' in feature]

In [291]:
final_df = pd.get_dummies(final_colnames)

In [292]:
final_df

,Alloc,CursorKind.ARRAY_SUBSCRIPT_EXPR,CursorKind.ASM_LABEL_ATTR,CursorKind.BINARY_OPERATOR,CursorKind.BREAK_STMT,CursorKind.CALL_EXPR,CursorKind.CASE_STMT,CursorKind.CHARACTER_LITERAL,CursorKind.CLASS_DECL,CursorKind.CLASS_TEMPLATE,...,CursorKind.UNEXPOSED_EXPR,CursorKind.UNION_DECL,CursorKind.USING_DECLARATION,CursorKind.USING_DIRECTIVE,CursorKind.VARIABLE_REF,CursorKind.VAR_DECL,CursorKind.VISIBILITY_ATTR,CursorKind.WHILE_STMT,SizeOf,WriteToPointer
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Creating the feature matrix:

In [281]:
def generate_features_matrix(ast_root):
    """
    Given a concretised & numbered clang ast, returns a matrix of one hot encoded features of node names kind and 
    whether it's alloc/writeToPointer/sizeOf/other, i.e. our feature matrix
    """
    index = []
    kind = {}
    spelling = {}

    def walk_tree_and_set_properties(node):
        out_degree = len(node.children)
        in_degree = 1
        degree = out_degree + in_degree
        
        index.append(node.identifier)
        
        kind[node.identifier] = node.kind
        spelling[node.identifier] = node.spelling
        
        if str(node.spelling) in writeToPointer_list:
            spelling[node.identifier] = 'writeToPointer'
        
        elif str(node.spelling) in sizeOf_list:
            spelling[node.identifier] = 'sizeOf'
            
        elif str(node.spelling) in alloc_list:
            spelling[node.identifier] = 'alloca'
        
        else:
            spelling[node.identifier] = ''
        

        for child in node.children:
            walk_tree_and_set_properties(child)

    walk_tree_and_set_properties(ast_root)
    
#     return spelling
    
    d = {'identifier': index, 'kind': list(kind.values()), 'spelling': list(spelling.values())}
        
    final_df = pd.DataFrame(data = d)
    final_df = final_df.set_index('identifier')
    
    dum_df = pd.get_dummies(final_df, prefix=['kind', 'spelling'])

    return dum_df.values

In [ ]:
def attempt(ast_root):
    """
    Given a concretised & numbered clang ast, returns a matrix of one hot encoded features of node names kind and 
    whether it's alloc/writeToPointer/sizeOf/other, i.e. our feature matrix
    """
    index = []
    kind = {}
    spelling = {}

    def walk_tree_and_set_properties(node):
        out_degree = len(node.children)
        in_degree = 1
        degree = out_degree + in_degree
        
        index.append(node.identifier)
        
        kind[node.identifier] = node.kind
        spelling[node.identifier] = node.spelling
        
        if str(node.spelling) in writeToPointer_list:
            spelling[node.identifier] = 'writeToPointer'
        
        elif str(node.spelling) in sizeOf_list:
            spelling[node.identifier] = 'sizeOf'
            
        elif str(node.spelling) in alloc_list:
            spelling[node.identifier] = 'alloca'
        
        else:
            spelling[node.identifier] = ''
        

        for child in node.children:
            walk_tree_and_set_properties(child)

    walk_tree_and_set_properties(ast_root)
    
#     return spelling
    
    d = {'identifier': index, 'kind': list(kind.values()), 'spelling': list(spelling.values())}
        
    final_df = pd.DataFrame(data = d)
    final_df = final_df.set_index('identifier')
    
    dum_df = pd.get_dummies(final_df, prefix=['kind', 'spelling'])

    return dum_df.values

In [285]:
mat = ast_roots.apply(generate_features_matrix)

In [288]:
mat.iloc[1]

array([[1, 0, 0, ..., 0, 0, 1],
       [0, 1, 0, ..., 0, 0, 1],
       [0, 1, 0, ..., 0, 0, 1],
       ...,
       [0, 0, 0, ..., 0, 0, 1],
       [0, 0, 0, ..., 0, 0, 1],
       [0, 0, 0, ..., 0, 0, 1]], dtype=uint8)

Sam: I think in order to get a dataframe of the feature matrices where the matrices are all the same size we need to append each testcase_ID in ast_roots to an exisiting dataframe that has all the 'node.kind' names as well as 'alloc', 'writeToPointer', 'sizeOf' for columns which already exists under final_colnames. To convert this into a matrix just use .values to each dataframe and store this into a 'master' dataframe with all the matrices. This still needs to be applied to all the datapoints, as I have currently done this with 500 datapoints. 